In [70]:
import pickle
import nltk
from nltk.corpus import stopwords
import pandas as pd
from scipy.spatial.distance import jensenshannon

In [2]:
nltk.download("stopwords")
stop_words=set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hrenduchinta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# import math
# import random
# import time
# from tqdm.auto import tqdm
# import submodlib
# import faiss
# import pickle
# import numpy as np
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import cosine_similarity
# import faiss
# from helper_fns import taylor_softmax_v1
# import torch
# import os
# from scipy.special import softmax
# from datasets import load_from_disk

/opt/conda/envs/ingenious/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# X=np.load("representations.npy")
# # X=np.load("wikitext-103-glove-embeddings.npy")
# N=X.shape[0]
# idx=np.random.randint(0, N, size=10000).tolist()
# representations=X[idx]
# # faiss.normalize_L2(representations)
# M=representations.shape[0]
# # dist_mat=euclidean_distances(representations)
# # data_sijs=np.exp(-dist_mat/dist_mat.mean())
# data_sijs=cosine_similarity(representations)
# obj = submodlib.FacilityLocationFunction(n = representations.shape[0], separate_rep=False, mode = 'dense', sijs = data_sijs)
# greedyList=obj.maximize(budget=int(0.25*M), optimizer="LazyGreedy", stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False, show_progress=True)
# subset_indices=[p[0] for p in greedyList]

[||||||||||||||||||||]100% [Iteration 2500 of 2500]      ]21% [Iteration 525 of 2500]

In [27]:
# dataset=load_from_disk("wikitext-103-processed-first-sentences")
# dataset=dataset["train"]
# full_partition_freq_dict={}
# pbar=tqdm(range(len(idx)))
# for i in idx:
#     for id in dataset[i]["input_ids"]:
#         if id in full_partition_freq_dict:
#             full_partition_freq_dict[id]+=1
#         else:
#             full_partition_freq_dict[id]=1
#     pbar.update(1)
# with open("full_partition_freqs.pkl", "wb") as f:
#     pickle.dump(full_partition_freq_dict, f)
# subset_freq_dict={}
# pbar=tqdm(range(len(subset_indices)))
# for i in subset_indices:
#     for id in dataset[idx[i]]["input_ids"]:
#         if id in subset_freq_dict:
#             subset_freq_dict[id]+=1
#         else:
#             subset_freq_dict[id]=1
#     pbar.update(1)
# with open("subset_freqs.pkl", "wb") as f:
#     pickle.dump(subset_freq_dict, f)

100%|██████████| 10000/10000 [00:14<00:00, 692.70it/s]


In [6]:
with open("vocab.txt", "r") as f:
    vocabulary=f.readlines()

In [13]:
def get_freqs(file_path):
    with open(file_path, "rb") as f:
        d=pickle.load(f)
        d={vocabulary[int(k)].strip():v for k, v in d.items() if ((int(k)>=1996 and int(k)<=29611) and (vocabulary[int(k)].strip() not in stop_words))}
        d={k:v for k, v in sorted(d.items(), key=lambda x: -x[1])}
    return d

In [75]:
files=["fulldata-freqdict.pkl", "ingenious-arr.pkl", "lazygreedy-random.pkl", "full_partition_freqs.pkl", "subset_freqs.pkl"]
data=[]
for file in files:
    d=get_freqs(file)
    data.append(d)
    csv_name=f"{file[:-4]}.csv"
    with open(csv_name, "w") as f:
        f.write("word,count\n")
        for k, v in d.items():
            f.write(f"{k},{v}\n")

In [76]:
df=pd.DataFrame.from_dict(data[:3]).transpose().fillna(0)
df.columns=["full-data", "ingenious-arr", "lazygreedy"]
for col in df.columns:
    new_name=f"{col}-frac"
    df[new_name]=df[col]/df[col].sum()

In [78]:
df["full-data-frac"]

category    1.070555e-02
##s         7.760591e-03
one         3.030728e-03
also        2.984038e-03
first       2.658172e-03
                ...     
##₀         1.738315e-08
##₄         1.587157e-08
h₂o         1.209262e-08
##ᵢ         6.802100e-09
##ₙ         1.511578e-09
Name: full-data-frac, Length: 27471, dtype: float64

In [79]:
jensenshannon(df["full-data-frac"], df["ingenious-arr-frac"])

0.003150739504721935